# Quick Start Tutorial: Feature Management

## Learning Objectives

In this tutorial you will learn:
1. How to view the lineage of a feature
2. How to manage the readiness of a feature
3. How to be informed of the readiness of a feature list
4. How to manage the status of a feature list
5. How FeatureByte deployment guardrails work
6. How to check the feature job status
7. How to manage versioning
8. How to disable deployment

## Set up the prerequisites

Learning Objectives

In this section you will:
* start your local featurebyte server
* import libraries
* learn about catalogs
* activate a pre-built catalog

In [1]:
# library imports
import pandas as pd
import numpy as np
import random

# load the featurebyte SDK
import featurebyte as fb

print("FeatureByte version " + fb.version)

# start the local server, then wait for it to be healthy before proceeding
fb.playground()

2023-03-27 19:52:04.005 | INFO     | featurebyte.docker.manager:start_playground:305 | Starting featurebyte service | {}


FeatureByte version 0.1.4


2023-03-27 19:52:12.166 | INFO     | featurebyte.docker.manager:start_playground:307 | Starting local spark service | {}
2023-03-27 19:52:19.375 | INFO     | featurebyte.docker.manager:start_playground:310 | Starting documentation service | {}
2023-03-27 19:52:26.369 | INFO     | featurebyte.docker.manager:start_playground:314 | Creating local spark feature store | {}
2023-03-27 19:52:26.902 | INFO     | featurebyte.docker.manager:start_playground:336 | Dataset grocery already exists, skipping import | {}
2023-03-27 19:52:26.902 | INFO     | featurebyte.docker.manager:start_playground:336 | Dataset healthcare already exists, skipping import | {}
2023-03-27 19:52:26.902 | INFO     | featurebyte.docker.manager:start_playground:336 | Dataset creditcard already exists, skipping import | {}


### Create a pre-built catalog for this tutorial, with the data, metadata, and features already set up

Note that creating a pre-built catalog is not a step you will do in real-life. This is a function specific to this quick-start tutorial to quickly skip over many of the preparatory steps and get you to a point where you can materialize features.

In a real-life project you would do data modeling, declaring the tables, entities, and the associated metadata. This would not be a frequent task, but forms the basis for best-practice feature engineering.

In [2]:
# get the functions to create a pre-built catalog
from prebuilt_catalogs import *

# create a new catalog for this tutorial
catalog_name = create_tutorial_catalog(PrebuiltCatalog.QuickStartFeatureManagement)

Cleaning up any existing tutorial catalogs
Building a quick start catalog for feature management named [quick start feature management 20230327:1952]
Creating new catalog
Catalog created
Registering the source tables
Registering the entities
Tagging the entities to columns in the data tables
Populating the feature store with example features
Saving Feature(s) |████████████████████████████████████████| 5/5 [100%] in 0.0s (115.28/s)                              
Loading Feature(s) |████████████████████████████████████████| 5/5 [100%] in 1.0s (5.02/s)                               
Saving Feature(s) |████████████████████████████████████████| 4/4 [100%] in 0.0s (117.81/s)                              
Loading Feature(s) |████████████████████████████████████████| 4/4 [100%] in 0.7s (5.81/s)                               
Saving Feature(s) |████████████████████████████████████████| 3/3 [100%] in 1.2s (2.42/s)                                
Loading Feature(s) |███████████████████████████████

### Example: Activate an existing catalog

In [3]:
# you can activate an existing catalog
catalog = fb.Catalog.activate(catalog_name)

## Manage Feature Readiness

Learning Objectives

In this section you will learn:
* how to change the readiness of a feature
* the meaning of each readiness value

### Feature readiness

A feature's readiness indicate a feature's suitability for use in feature lists.

DEPRECATED: Indicates that the feature version is not recommended for either training or online serving.<br>
QUARANTINE: Indicates that the feature version has recently had some issues and should be used with caution. It is currently under review.<br>
DRAFT: The feature is being prototyped and is not yet ready for production.<br>
PRODUCTION_READY: Indicates that the feature version is ready to be used in production.<br>

In [4]:
# list the features in the catalog - note the readiness of each
catalog.list_features()

,name,dtype,readiness,online_enabled,tables,primary_tables,entities,primary_entities,created_at
0,InvoiceUniqueProductGroups,OBJECT,DEPRECATED,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-03-27 11:53:24.818
1,InvoiceUniqueProductGroupCount,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-03-27 11:53:19.841
2,InvoiceDiscountAmount,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-03-27 11:53:18.269
3,InvoiceItemCount,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-03-27 11:53:17.043
4,CustomerYearOfBirth,INT,PRODUCTION_READY,True,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[grocerycustomer],[grocerycustomer],2023-03-27 11:53:15.747
5,CustomerSpend_14d,FLOAT,PRODUCTION_READY,True,[GROCERYINVOICE],[GROCERYINVOICE],[grocerycustomer],[grocerycustomer],2023-03-27 11:53:15.143
6,CustomerInventory_24w,OBJECT,PRODUCTION_READY,True,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-03-27 11:53:13.906
7,CustomerInventory_28d,OBJECT,PRODUCTION_READY,True,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-03-27 11:53:12.487
8,StateMeanLongitude,FLOAT,DRAFT,False,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[frenchstate],[frenchstate],2023-03-27 11:53:11.228
9,StateMeanLatitude,FLOAT,DRAFT,False,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[frenchstate],[frenchstate],2023-03-27 11:53:10.657


### Example: Set features to production ready

In [5]:
# change the state features to be production ready
for feature_name in catalog.list_features().name:
    feature = catalog.get_feature(feature_name)
    
    # does the feature name contain the word "state"?
    if "State" in feature.name:
        feature.update_readiness("PRODUCTION_READY")

In [6]:
# list the features in the catalog - note the readiness of each
catalog.list_features()

,name,dtype,readiness,online_enabled,tables,primary_tables,entities,primary_entities,created_at
0,InvoiceUniqueProductGroups,OBJECT,DEPRECATED,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-03-27 11:53:24.818
1,InvoiceUniqueProductGroupCount,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-03-27 11:53:19.841
2,InvoiceDiscountAmount,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-03-27 11:53:18.269
3,InvoiceItemCount,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-03-27 11:53:17.043
4,CustomerYearOfBirth,INT,PRODUCTION_READY,True,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[grocerycustomer],[grocerycustomer],2023-03-27 11:53:15.747
5,CustomerSpend_14d,FLOAT,PRODUCTION_READY,True,[GROCERYINVOICE],[GROCERYINVOICE],[grocerycustomer],[grocerycustomer],2023-03-27 11:53:15.143
6,CustomerInventory_24w,OBJECT,PRODUCTION_READY,True,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-03-27 11:53:13.906
7,CustomerInventory_28d,OBJECT,PRODUCTION_READY,True,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-03-27 11:53:12.487
8,StateMeanLongitude,FLOAT,PRODUCTION_READY,False,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[frenchstate],[frenchstate],2023-03-27 11:53:11.228
9,StateMeanLatitude,FLOAT,PRODUCTION_READY,False,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[frenchstate],[frenchstate],2023-03-27 11:53:10.657


## Manage Feature List Status

Learning Objectives

In this section you will learn:
* how to change the status of a feature list
* the meaning of each status value
* how to deploy a feature list

### Feature list status

A feature list's status indicate its suitability for use in production.

DEPRECATED: The feature list is no longer suitable for production use.<br>
DRAFT: The feature list is being prototyped, and is not yet suitable for production.<br>
PUBLIC_DRAFT: The feature list is ready for review and sharing, but is not yet in production.<br>
PUBLISHED: The feature list has been deployed into production.<br>

In [7]:
# list the feature lists in the catalog - note the status of each
# Note the readiness fraction which represents the proportion of features that are production ready
# Note the online fraction which represents the proportion of features that are being used in production
catalog.list_feature_lists()

,name,num_features,status,deployed,readiness_frac,online_frac,tables,entities,created_at
0,InvoiceFeatureList,3,DRAFT,False,0.0,0.0,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[groceryinvoice],2023-03-27 11:53:25.735
1,CustomerFeatureList,4,PUBLISHED,True,1.0,1.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...",[grocerycustomer],2023-03-27 11:53:23.086
2,StateFeatureList,5,DRAFT,False,1.0,0.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...",[frenchstate],2023-03-27 11:53:21.121


### Example: Make a feature list public

When a feature list is reviewed and ready for sharing with other users, change its status to PUBLIC_DRAFT

In [8]:
# get the state feature list
state_feature_list = catalog.get_feature_list("StateFeatureList")

# update the status to public draft
state_feature_list.update_status("PUBLIC_DRAFT")

Loading Feature(s) |████████████████████████████████████████| 5/5 [100%] in 1.0s (4.83/s)                               


In [9]:
# list the feature lists in the catalog - note the status of each
catalog.list_feature_lists()

,name,num_features,status,deployed,readiness_frac,online_frac,tables,entities,created_at
0,InvoiceFeatureList,3,DRAFT,False,0.0,0.0,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[groceryinvoice],2023-03-27 11:53:25.735
1,CustomerFeatureList,4,PUBLISHED,True,1.0,1.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...",[grocerycustomer],2023-03-27 11:53:23.086
2,StateFeatureList,5,PUBLIC_DRAFT,False,1.0,0.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...",[frenchstate],2023-03-27 11:53:21.121


### Example: Deploy a feature list

Deploying a feature list changes its status to published

In [10]:
# deploy the state feature list
state_feature_list.deploy(enable=True)

# set the feature status to published
state_feature_list.update_status("PUBLISHED")

Loading Feature(s) |████████████████████████████████████████| 5/5 [100%] in 1.0s (5.22/s)                               
Done! |████████████████████████████████████████| 100% in 36.5s (2.74%/s)                                                


In [11]:
# list the feature lists in the catalog - note the status of each
catalog.list_feature_lists()

,name,num_features,status,deployed,readiness_frac,online_frac,tables,entities,created_at
0,InvoiceFeatureList,3,DRAFT,False,0.0,0.0,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[groceryinvoice],2023-03-27 11:53:25.735
1,CustomerFeatureList,4,PUBLISHED,True,1.0,1.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...",[grocerycustomer],2023-03-27 11:53:23.086
2,StateFeatureList,5,PUBLISHED,True,1.0,1.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...",[frenchstate],2023-03-27 11:53:21.121


## Versioning

Learning Objectives

In this section you will learn:
* about feature versions and feature list versions
* how to change the table cleaning operations for a feature
* how to change the feature job settings for a feature
* how to manage the default versions for a feature or a feature list
* how to create new versions of features and feature lists

### Concept: Feature version
A Feature Version enables the reuse of a Feature with varying Feature Job settings or distinct cleaning operations.

If the availability or freshness of the data source change, new versions of the feature can be generated with a new Feature Job Setting. On the other hand, if changes occur in the data quality of the data sources, new versions of the feature can be created with new cleaning operations that address the new quality issues.

To ensure the seamless inference of Machine Learning tasks that depend on the feature, old versions of the feature can still be served without any disruption.

### Example: Get table cleaning operations for a feature

In [12]:
# get the InvoiceDiscountAmount feature
invoice_discount_amount = catalog.get_feature("InvoiceDiscountAmount")

# list the feature versions
display(invoice_discount_amount.list_versions())

,name,version,dtype,readiness,online_enabled,tables,primary_tables,entities,primary_entities,created_at
0,InvoiceDiscountAmount,V230327,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-03-27 11:53:18.256


In [13]:
# No cleaning has been operated on this feature
invoice_discount_amount.info()['table_cleaning_operation']

{'this': [], 'default': []}

### Example: Updating table cleaning operations for a feature

In [14]:
# update the data cleaning operations in the InvoiceDiscountAmount feature
new_version = invoice_discount_amount.create_new_version(
  table_cleaning_operations=[
    fb.TableCleaningOperation(
      table_name="INVOICEITEMS",
      column_cleaning_operations=[
        fb.ColumnCleaningOperation(
          column_name="Discount",
          cleaning_operations=[
              fb.MissingValueImputation(imputed_value=0.0),
              fb.ValueBeyondEndpointImputation(type="less_than", end_point=0, imputed_value=None)
          ],
        )
      ],
    )
  ]
)

In [15]:
# list the feature versions
feature_versions = invoice_discount_amount.list_versions()

# sort by created_at ascending
feature_versions.sort_values(by="created_at", ascending=True, inplace=True)

# display only the InvoiceUniqueProductGroups feature - note the new version that has been created
display(feature_versions)

,name,version,dtype,readiness,online_enabled,tables,primary_tables,entities,primary_entities,created_at
1,InvoiceDiscountAmount,V230327,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-03-27 11:53:18.256
0,InvoiceDiscountAmount,V230327_1,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-03-27 11:55:43.213


In [16]:
# Check new cleaning info
new_version.info()['table_cleaning_operation']['this']

[{'table_name': 'INVOICEITEMS',
  'column_cleaning_operations': [{'column_name': 'Discount',
    'cleaning_operations': [{'imputed_value': 0, 'type': 'missing'},
     {'imputed_value': None, 'type': 'less_than', 'end_point': 0}]}]}]

In [17]:
# Check feature definition file
new_version.definition

'# Generated by SDK version: 0.1.4\nfrom bson import ObjectId\nfrom featurebyte import ColumnCleaningOperation\nfrom featurebyte import ItemTable\nfrom featurebyte import MissingValueImputation\nfrom featurebyte import ValueBeyondEndpointImputation\n\n\n# item_table name: "INVOICEITEMS", event_table name: "GROCERYINVOICE"\nitem_table = ItemTable.get_by_id(ObjectId("6421839ac15414bd69da9a7f"))\nitem_view = item_table.get_view(\n    event_suffix=None,\n    view_mode="manual",\n    drop_column_names=[],\n    column_cleaning_operations=[\n        ColumnCleaningOperation(\n            column_name="Discount",\n            cleaning_operations=[\n                MissingValueImputation(imputed_value=0),\n                ValueBeyondEndpointImputation(\n                    type="less_than", end_point=0, imputed_value=None\n                ),\n            ],\n        )\n    ],\n    event_drop_column_names=["record_available_at"],\n    event_column_cleaning_operations=[],\n    event_join_column_names=[\n        "Timestamp",\n        "GroceryInvoiceGuid",\n        "GroceryCustomerGuid",\n    ],\n)\nfeat = item_view.groupby(\n    by_keys=["GroceryInvoiceGuid"], category=None\n).aggregate(\n    value_column="Discount",\n    method="sum",\n    feature_name="InvoiceDiscountAmount",\n    skip_fill_na=True,\n)\noutput = feat\n'

### Concept: Default feature version
    
The default version of a feature streamlines the process of reusing features by providing the most appropriate version. Additionally, it simplifies the creation of new versions of feature lists.

By default, the feature's version with the highest level of readiness is considered, unless the user overrides this selection. In cases where multiple versions share the highest level of readiness, the most recent version is automatically chosen as the default.

When a feature is accessed without specifying a version ID but only by its name, the default version is automatically retrieved.

### Example: The version we just created should be the default as no other version has a higher readiness and it is the latest version.

In [18]:
new_version.is_default

True

### Example: Get the feature job settings for a feature

Note that changing feature job settings will only affect time-aware features e.g. features created using aggregate_over. It will not affect features based upon simple aggregation.

In [19]:
# get the CustomerInventory_28d feature
customer_inventory_28d_feature = catalog.get_feature("CustomerInventory_28d")

# list the feature versions
display(customer_inventory_28d_feature.list_versions())

,name,version,dtype,readiness,online_enabled,tables,primary_tables,entities,primary_entities,created_at
0,CustomerInventory_28d,V230327,OBJECT,PRODUCTION_READY,True,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-03-27 11:53:12.471


In [20]:
# Get the feature job settings for CustomerInventory_28d
customer_inventory_28d_feature.info()['table_feature_job_setting']['this']

[{'table_name': 'GROCERYINVOICE',
  'feature_job_setting': {'blind_spot': '0s',
   'frequency': '3600s',
   'time_modulo_frequency': '90s'}}]

Note that the table name here is the event table associated with the item table as the timestamp originates from this table.

In [21]:
# show the feature job settings for the grocery invoice table
grocery_invoice_table = catalog.get_table("GROCERYINVOICE")
grocery_invoice_table.default_feature_job_setting

FeatureJobSetting(blind_spot='145', frequency='60m', time_modulo_frequency='90s')

### Example: Change the feature job settings for a feature

Note that changing feature job settings will only affect time-aware features e.g. features created using aggregate_over. It will not affect features based upon simple aggregation.

In [22]:
# update the data cleaning operations for the InvoiceDiscountAmount feature to be more conservative
new_version = customer_inventory_28d_feature.create_new_version(
  table_feature_job_settings=[
    fb.TableFeatureJobSetting(
      table_name="GROCERYINVOICE",
      feature_job_setting=fb.FeatureJobSetting(
        blind_spot="160s",
        frequency="60m",
        time_modulo_frequency="90s",
      )
    ),
  ]
)

In [23]:
# list the feature versions
feature_versions = customer_inventory_28d_feature.list_versions()

# sort by created_at ascending
feature_versions.sort_values(by="created_at", ascending=True, inplace=True)

# note that the new version is a draft, and that the old version remains production ready
display(feature_versions)

,name,version,dtype,readiness,online_enabled,tables,primary_tables,entities,primary_entities,created_at
1,CustomerInventory_28d,V230327,OBJECT,PRODUCTION_READY,True,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-03-27 11:53:12.471
0,CustomerInventory_28d,V230327_1,OBJECT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-03-27 11:55:51.723


In [24]:
# Get the feature job settings for the new version
new_version.info()['table_feature_job_setting']['this']

[{'table_name': 'GROCERYINVOICE',
  'feature_job_setting': {'blind_spot': '160s',
   'frequency': '3600s',
   'time_modulo_frequency': '90s'}}]

In [25]:
# Check feature definition file
new_version.definition

'# Generated by SDK version: 0.1.4\nfrom bson import ObjectId\nfrom featurebyte import DimensionTable\nfrom featurebyte import ItemTable\n\n\n# item_table name: "INVOICEITEMS", event_table name: "GROCERYINVOICE"\nitem_table = ItemTable.get_by_id(ObjectId("6421839ac15414bd69da9a7f"))\nitem_view = item_table.get_view(\n    event_suffix=None,\n    view_mode="manual",\n    drop_column_names=[],\n    column_cleaning_operations=[],\n    event_drop_column_names=["record_available_at"],\n    event_column_cleaning_operations=[],\n    event_join_column_names=[\n        "Timestamp",\n        "GroceryInvoiceGuid",\n        "GroceryCustomerGuid",\n    ],\n)\n\n# dimension_table name: "GROCERYPRODUCT"\ndimension_table = DimensionTable.get_by_id(ObjectId("6421839cc15414bd69da9a80"))\ndimension_view = dimension_table.get_view(\n    view_mode="manual", drop_column_names=[], column_cleaning_operations=[]\n)\nitem_view.join(dimension_view, on=None, how="left", rsuffix="")\ngrouped = item_view.groupby(\n    by_keys=["GroceryCustomerGuid"], category="ProductGroup"\n).aggregate_over(\n    value_column=None,\n    method="count",\n    windows=["28d"],\n    feature_names=["CustomerInventory_28d"],\n    feature_job_setting={\n        "blind_spot": "160s",\n        "frequency": "3600s",\n        "time_modulo_frequency": "90s",\n    },\n    skip_fill_na=True,\n)\nfeat = grouped["CustomerInventory_28d"]\noutput = feat\n'

### Example: Change default feature version mode

The new version of CustomerInventory_28d is not the default as this version is a draft and the prior version is production ready.

In [26]:
new_version.is_default

False

The default can be changed only if the default version mode is set as manual.

In [27]:
# guardrail if default version mode is not MANUAL
# the new version cannot be set as the default
try:
    new_version.as_default_version()
except Exception as ex:
    print(ex)

Cannot set default feature ID when default version mode is not MANUAL


In [28]:
# change mode to manual and set new version as manual
new_version.update_default_version_mode("MANUAL")
new_version.as_default_version()

In [29]:
# check if new version is default
new_version.is_default

True

In [30]:
# check if version that is obtained by default is the version we just created
customer_inventory_28d_feature = catalog.get_feature("CustomerInventory_28d")
customer_inventory_28d_feature.version

VersionIdentifier(name='V230327', suffix=1)

### Example: Cannot have more than one production ready version of a feature

In [31]:
# change the readiness of the new version of CustomerInventory_28d to production ready
try:
    new_version.update_readiness("PRODUCTION_READY")
except Exception as ex:
    print("Error changing the readiness of the new version to production ready")
    print(ex)

Error changing the readiness of the new version to production ready
Found another feature version that is already PRODUCTION_READY. Please deprecate the feature CustomerInventory_28d with ID 642183a7c15414bd69da9a93 first before promoting the promoted version as there can only be one feature version that is production ready at any point in time. We are unable to promote the feature with ID 6421844702047323e0343901 right now.


### Example: Create version of a Feature List

The Feature List Version allows the use of the latest version of each feature. Upon creation of a new feature list version, the latest default versions of features are employed, unless particular feature versions are specified.

In [32]:
# the current default of the feature list has all feature versions production ready
customer_feature_list = catalog.get_feature_list("CustomerFeatureList")
customer_feature_list.list_features()

Loading Feature(s) |████████████████████████████████████████| 4/4 [100%] in 0.9s (4.44/s)                               


,name,version,dtype,readiness,online_enabled,tables,primary_tables,entities,primary_entities,created_at
0,CustomerYearOfBirth,V230327,INT,PRODUCTION_READY,True,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[grocerycustomer],[grocerycustomer],2023-03-27 11:53:15.736
1,CustomerSpend_14d,V230327,FLOAT,PRODUCTION_READY,True,[GROCERYINVOICE],[GROCERYINVOICE],[grocerycustomer],[grocerycustomer],2023-03-27 11:53:15.127
2,CustomerInventory_24w,V230327,OBJECT,PRODUCTION_READY,True,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-03-27 11:53:13.891
3,CustomerInventory_28d,V230327,OBJECT,PRODUCTION_READY,True,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-03-27 11:53:12.471


In [33]:
# create new version with the new default of CustomerInventory_28d
new_feature_list_version = customer_feature_list.create_new_version(mode="auto")

Loading Feature(s) |████████████████████████████████████████| 4/4 [100%] in 0.7s (5.57/s)                               


In [34]:
new_feature_list_version.list_features()

,name,version,dtype,readiness,online_enabled,tables,primary_tables,entities,primary_entities,created_at
0,CustomerInventory_28d,V230327_1,OBJECT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-03-27 11:55:51.723
1,CustomerYearOfBirth,V230327,INT,PRODUCTION_READY,True,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[grocerycustomer],[grocerycustomer],2023-03-27 11:53:15.736
2,CustomerSpend_14d,V230327,FLOAT,PRODUCTION_READY,True,[GROCERYINVOICE],[GROCERYINVOICE],[grocerycustomer],[grocerycustomer],2023-03-27 11:53:15.127
3,CustomerInventory_24w,V230327,OBJECT,PRODUCTION_READY,True,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-03-27 11:53:13.891


In [35]:
# check readiness
new_feature_list_version.info()['production_ready_fraction']

{'this': 0.75, 'default': 1.0}

### Default version of a Feature List

The default version of a feature list is the version with the highest fraction of production ready features.

The new version is not the default as its production_ready_fraction is lower than the prior version for CustomerInventory_28d.

In [36]:
# the new version is not the default as it is production_ready_fraction is lower than the current default
new_feature_list_version.is_default

False

## Deployment Guardrails

FeatureByte has guardrails for deployment, and will prevent you from deploying a feature list that has features that are not production ready.

Learning Objectives

In this section you will learn:
* how to check the readiness of a feature list
* how to deploy a feature list

### Example: Check readiness of a feature list

The Feature List Readiness metric provides information to users about the readiness status of a Feature List. This metric represents the percentage of features that are production ready within the given feature list.

In [37]:
# get the invoice feature list
invoice_feature_list = catalog.get_feature_list("InvoiceFeatureList")

# check feature list is ready to be deployed
invoice_feature_list.info()['production_ready_fraction']

Loading Feature(s) |████████████████████████████████████████| 3/3 [100%] in 0.7s (4.47/s)                               


{'this': 0.0, 'default': 0.0}

In [38]:
invoice_feature_list.list_features()

,name,version,dtype,readiness,online_enabled,tables,primary_tables,entities,primary_entities,created_at
0,InvoiceUniqueProductGroups,V230327,OBJECT,DEPRECATED,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-03-27 11:53:24.806
1,InvoiceDiscountAmount,V230327,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-03-27 11:53:18.256
2,InvoiceItemCount,V230327,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-03-27 11:53:17.029


### Example: Deploy a feature list when production readiness is not 100%

In [39]:
# deploy the invoice feature list
try:
    invoice_feature_list.deploy(enable=True)
except Exception as ex:
    print("Error deploying the invoice feature list")
    print(ex)

Loading Feature(s) |████████████████████████████████████████| 3/3 [100%] in 0.6s (5.08/s)                               
Working... |⚠︎                                       | (!) 0% in 3.3s (0.00%/s)                                          
Error deploying the invoice feature list
Traceback (most recent call last):
  File "/app/.venv/lib/python3.8/site-packages/celery/app/trace.py", line 451, in trace_task
    R = retval = fun(*args, **kwargs)
  File "/app/.venv/lib/python3.8/site-packages/featurebyte/worker/__init__.py", line 53, in __call__
    return loop.run_until_complete(self._run(*args, **kwargs))
  File "/usr/local/lib/python3.8/asyncio/base_events.py", line 616, in run_until_complete
    return future.result()
  File "/app/.venv/lib/python3.8/site-packages/featurebyte/worker/__init__.py", line 48, in _run
    return await result
  File "/app/.venv/lib/python3.8/site-packages/featurebyte/worker/task_executor.py", line 61, in execute_task
    await executor.execute()
  File "/

In [40]:
# show the feature lists - note that the invoice feature list has not been deployed
catalog.list_feature_lists()

,name,num_features,status,deployed,readiness_frac,online_frac,tables,entities,created_at
0,InvoiceFeatureList,3,DRAFT,False,0.0,0.0,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[groceryinvoice],2023-03-27 11:53:25.735
1,CustomerFeatureList,4,PUBLISHED,True,1.0,1.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...",[grocerycustomer],2023-03-27 11:53:23.086
2,StateFeatureList,5,PUBLISHED,True,1.0,1.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...",[frenchstate],2023-03-27 11:53:21.121


## Disable Deployment

Learning Objectives

In this section you will learn:
* how to disable deployments

### Example: Disabling a deployment

In [41]:
# disable the deployments
catalog.get_feature_list("CustomerFeatureList").deploy(enable=False)
catalog.get_feature_list("StateFeatureList").deploy(enable=False)

# show the feature lists status
catalog.list_feature_lists()

Loading Feature(s) |████████████████████████████████████████| 4/4 [100%] in 0.8s (5.24/s)                               
Loading Feature(s) |████████████████████████████████████████| 4/4 [100%] in 1.1s (3.56/s)                               
Done! |████████████████████████████████████████| 100% in 15.4s (6.49%/s)                                                
Loading Feature(s) |████████████████████████████████████████| 5/5 [100%] in 0.9s (5.43/s)                               
Loading Feature(s) |████████████████████████████████████████| 5/5 [100%] in 1.0s (4.97/s)                               
Done! |████████████████████████████████████████| 100% in 12.3s (8.12%/s)                                                


,name,num_features,status,deployed,readiness_frac,online_frac,tables,entities,created_at
0,InvoiceFeatureList,3,DRAFT,False,0.0,0.0,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[groceryinvoice],2023-03-27 11:53:25.735
1,CustomerFeatureList,4,PUBLISHED,False,1.0,0.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...",[grocerycustomer],2023-03-27 11:53:23.086
2,StateFeatureList,5,PUBLISHED,False,1.0,0.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...",[frenchstate],2023-03-27 11:53:21.121


## Next Steps

Now that you've completed the quick-start feature management tutorial, you can put your knowledge into practice or learn more:<br>
1) Put your knowledge into practice by creating features in the "credit card dataset feature engineering playground" or "healthcare dataset feature engineering playground" catalogs
2) Learn more about feature engineering via the "Deep Dive Feature Engineering" tutorial
3) Learn about data modeling via the "Deep Dive Data Modeling" tutorial